In [26]:
import pandas as pd

import json


In [33]:
questions_raw = pd.read_csv('./data/questions_details_2.csv')

In [38]:
questions = questions_raw.copy()
# creator column is a list of strings. I want to create a new row for each creator
questions['creator'] = questions['creator'].apply(lambda x: eval(x) if isinstance(x, str) else x)
questions = questions.explode('creator')
questions = questions.drop_duplicates()



In [77]:
# use the document_date to create a new column based on YEAER-MONTH
questions['year-month'] = questions['document_date'].apply(lambda x: '-'.join(x.split('-')[:2]))
questions['member_id'] = questions['creator'].apply(lambda x: str(x).split('/')[-1] if 'person' in str(x) else None)

questions = questions[~questions['member_id'].isna()]


In [99]:
df_q = questions.pivot_table(index='member_id', columns='year-month', values='id', aggfunc='size', fill_value=0)

df_q['total'] = df_q.sum(axis=1)

df_q = df_q.sort_values(by='total', ascending=False).fillna(0)

del df_q['total']

# set index to be integer
df_q.index = df_q.index.astype(float)


df_q.head()

year-month,2008-06,2009-01,2009-02,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,...,2024-03,2024-04,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02
member_id,,,,,,,,,,,,,,,,,,,,,
97198.0,0,0,0,12,5,8,31,57,16,55,...,0,0,0,0,0,0,0,0,0,0
125012.0,0,0,0,0,0,3,3,2,5,5,...,0,0,0,0,1,9,1,0,4,1
118858.0,0,0,0,5,5,45,44,52,43,55,...,0,0,0,0,0,0,0,0,0,0
96706.0,0,0,0,13,2,12,16,17,13,13,...,0,0,0,0,0,0,0,0,0,0
124835.0,0,0,0,1,0,3,8,4,9,5,...,7,4,0,0,0,0,0,0,0,0


# Meetings

In [58]:
df_mathces = pd.read_csv('./high_confidence_matches.csv')
df_meetings = pd.read_csv('./data/meetings.csv')
df_register = pd.read_excel(
    r"D:\OneDrive\UFPR\00.PPGCP\00. Doutorado\Tese\Base de dados\Transparency register\Organisations in Transparency Register-2024-JAN.xls"
)

In [59]:
df_combined = df_mathces.merge(df_register, left_on='matched_organization', right_on='Name')

In [60]:
cat_mapper = {
    "Non-governmental organisations, platforms and networks and similar": "NGOs",
    "Trade and business associations": "Business",
    "Companies & groups": "Business",
    "Other organisations, public or mixed entities": "Other",
    "Trade unions and professional associations": "Other",
    "Self-employed individuals": "Other",
    "Think tanks and research institutions": "Other",
    "Professional consultancies": "Other",
    "Academic institutions": "Other",
    "Associations and networks of public authorities": "Other",
    "Organisations representing churches and religious communities": "Other",
    "Law firms": "Other",
    "Entities, offices or networks established by third countries": "Other",
}


df_combined["month"] = df_combined["meeting_date"].apply(lambda x: "-".join(x.split("-")[:2]))
df_combined["cat_mapped"] = df_combined["Category of registration"].apply(
    lambda x: cat_mapper.get(x)
)

In [79]:
df_m = df_combined.pivot_table(index='member_id', columns="month", aggfunc='size', fill_value=0)

# save

In [107]:
df_q.to_csv('./data/silver/df_questions_by_period.csv')
df_m.to_csv('./data/silver/df_meetings_by_period.csv')
